In [34]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, Dropout, Subtract, \
                        Flatten, MaxPooling2D, Multiply, Lambda, Add, Dot
from keras.backend import constant
from keras import optimizers

from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.constraints import max_norm
import keras.backend as K

###
# GMB dSt = mu*St*dt+sigma*St*dBt
###

N = 30
dt = 1/30
eps = 0.01
mu = 0.1
vol = 0.2

layers = []
for _ in range(N):
    layer1 = Dense(8, activation='tanh',
                      kernel_initializer=initializers.RandomNormal(0,1),
                      bias_initializer='random_normal')
    layer2 = Dense(1, activation='tanh',
                      kernel_initializer=initializers.RandomNormal(0,1),
                      bias_initializer='random_normal')
    layers += [[layer1,layer2]]
    
price = Input(shape=(1,))
wealth = Input(shape=(1,))
strategy = Input(shape=(1,))

inputs = [price, wealth, strategy]

for i in range(N):
    helper = Concatenate()([price,strategy])
    helper1 = layers[i][0](helper)
    strategyHelper = layers[i][1](helper1)
    
    strategyChange = Subtract()([strategy,strategyHelper])
    strategyChange = Lambda(lambda x : K.abs(x))(strategyChange)
    wealth = Add()([wealth, Lambda(lambda x : -eps*x)(strategyChange)])
    strategy = strategyHelper
     
    BMinc = Input(shape=(1,))
    inputs += [BMinc]
    
    aux1 = Lambda(lambda x : mu*x*dt)(price)
    aux2 = Lambda(lambda x : vol*x)(price)
    aux3 = Multiply()([aux2,BMinc])
    priceInc = Add()([aux1,aux3])
    price = Add()([price,priceInc])
    aux = Multiply()([priceInc,strategyHelper])
    wealth = Add()([wealth,aux])
    state = wealth

model = Model(inputs = inputs, outputs = wealth)
    



In [35]:
def custom_loss(y_true, y_pred):
    return K.mean(K.exp(-0.5*y_pred))

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['accuracy'])

In [36]:
K = 10**4
import matplotlib.pyplot as plt

xtrain = ([1.0*np.ones(K)] +
          [1.0*np.ones(K)]+
          [np.zeros(K)]+
          [np.random.normal(0,np.sqrt(dt),K) for i in range(N)])
ytrain = np.zeros(shape = (K,1))

for i in range(5):
    model.fit(x=xtrain,y=ytrain, epochs=10, verbose=True)
    print(np.mean(model.predict(xtrain)))

Epoch 1/10
10000/10000 [==============================] - 13s 1ms/step - loss: 0.6402 - acc: 0.0119
Epoch 2/10
10000/10000 [==============================] - 5s 539us/step - loss: 0.6098 - acc: 0.0000e+00
Epoch 3/10
10000/10000 [==============================] - 5s 536us/step - loss: 0.6009 - acc: 0.0000e+00
Epoch 4/10
10000/10000 [==============================] - 6s 579us/step - loss: 0.5958 - acc: 0.0000e+00
Epoch 5/10
10000/10000 [==============================] - 6s 550us/step - loss: 0.5914 - acc: 0.0000e+00
Epoch 6/10
10000/10000 [==============================] - 5s 546us/step - loss: 0.5869 - acc: 0.0000e+00
Epoch 7/10
10000/10000 [==============================] - 5s 544us/step - loss: 0.5842 - acc: 0.0000e+00
Epoch 8/10
10000/10000 [==============================] - 6s 587us/step - loss: 0.5830 - acc: 1.0000e-04
Epoch 9/10
10000/10000 [==============================] - 5s 536us/step - loss: 0.5825 - acc: 1.0000e-04
Epoch 10/10
10000/10000 [==============================] - 5